In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
file_path = 'mathematics_complete_cleaned_data_mcq_v1.json'
naep_df =pd.read_json(file_path)
naep_df

,question_id,question,options,answer,student_scores
0,2013-4M3 #7 M145801,Which of these operations will give the smalle...,"{'A': 'Subtract 1 from 1,000.', 'B': 'Multiply...",D,"{'A': 0.194041272702227, 'B': 0.04355844031378..."
1,2013-4M3 #11 M146201,Lisa sold 15 cups of lemonade on Saturday and ...,"{'A': '15+15', 'B': '2×15', 'C': '15+(2×15)', ...",C,"{'A': 0.16055910107260202, 'B': 0.430579561175..."
2,2005-8M4 #9 M067801,Alba needed to know about how much the sum of ...,"{'A': '19, 23, 38', 'B': '19, 24, 38', 'C': '2...",C,"{'A': 0.046488682792707, 'B': 0.04396469065324..."
3,1992-8M5 #5 M022301,There are 50 hamburgers to serve 38 children. ...,"{'A': '6', 'B': '12', 'C': '26', 'D': '38'}",B,"{'A': 0.03181152512075, 'B': 0.921062799490369..."
4,1992-8M5 #12 M022901,By how much would 217 be increased if the digi...,"{'A': '4', 'B': '40', 'C': '44', 'D': '400'}",B,"{'A': 0.117117100821711, 'B': 0.72247827157345..."
...,...,...,...,...,...
244,2005-12M12 #17 M130101,Yvonne has studied the cost of tickets over ti...,"{'A': '$5', 'B': '$8', 'C': '$13', 'D': '$18',...",A,"{'A': 0.264290927804532, 'B': 0.07402657216099..."
245,1996-12M13 #3 M060101,A certain machine produces 300 nails per minut...,"{'A': '4 min', 'B': '4 min 6 sec', 'C': '4 min...",C,"{'A': 0.091148927618188, 'B': 0.17997796448888..."
246,1996-12M13 #5 M060301,Which of the following pieces of information w...,"{'A': 'Median income of drivers', 'B': 'Range ...",A,"{'A': 0.721247794961391, 'B': 0.10513433955746..."
247,1992-12M15 #5 M061201,"In a group of 1,200 adults, there are 300 vege...","{'A': '1 to 3', 'B': '1 to 4', 'C': '3 to 1', ...",C,"{'A': 0.07673334838676, 'B': 0.275444895635577..."


In [3]:
years_naep = ['2013', '2009', '2005']
question_list = []

for year in years_naep:
    for idx, row in naep_df.iterrows():
        if year in row['question_id']:
            question_list.append({'question_id': row['question_id'], 'question': row['question'], 'options': row['options'], 'answer': row['answer']})
            
        df = pd.DataFrame(question_list)
        df.to_csv(year + '_naep_questions.csv', index=False)

In [ ]:
import re

def extract_id(text):
    match = re.search(r'#\d+', text)
    if match:
        start_index = match.end()
        return text[start_index:].strip()
    else:
        return None

In [20]:
years_naep = ['2013', '2009', '2005']

question_list = []

for years in years_naep:
    print('Params for '+ years + ': ')
    df = pd.read_csv('naep data/questions/' + years + '_naep_questions.csv')
    irt_param = pd.read_csv('naep data/parameters/NAEP_' + years + '_total.csv')
    ids = irt_param['NAEP ID'].values

    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        q_id = extract_id(row['question_id'])
        if q_id in ids:
            a = irt_param[ irt_param['NAEP ID'] == q_id]['aj'].values[0]
            b = irt_param[ irt_param['NAEP ID'] == q_id]['bj'].values[0]

            question_list.append({'question_id': q_id, 'question': row['question'], 'options': row['options'], 'answer': row['answer'],
                                  'a': a, 'b': b})
            
    df_params = pd.DataFrame(question_list)
    df_params.to_csv('naep data/questions/' + years + '_naep_complete.csv', index=False)

Params for 2013: 


  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:00<00:00, 9802.64it/s]


Params for 2009: 


100%|██████████| 27/27 [00:00<00:00, 14050.40it/s]


Params for 2005: 


100%|██████████| 68/68 [00:00<00:00, 4187.90it/s]


### combine into one file

In [23]:
df2 = pd.read_csv('naep data/questions/2005_naep_complete.csv')
df1 = pd.read_csv('naep data/questions/NAEP_questions.csv')
df1 = df1[df1['image'] == 0]
df1 = df1.drop(columns=['image', 'category', 'difficulty'])
df1

,question_id,question,options,answer,a,b
0,M184601,Which of the following expressions is NOT equi...,A. (a+b)x+(a+b)y\nB. a(x+y)+b(x+y)\nC. (b+a)(y...,D,1.45,0.06
1,M130101,Yvonne has studied the cost of tickets over ti...,A. $5\nB. $8\nC. $13\nD. $18\nE. $26,A,2.46,1.24
2,M190101,The first four terms in a sequence are shown b...,A. The 5th term\nB. The 6th term\nC. The 7th t...,C,0.69,-0.09
9,M191501,John is going to cover an attic floor with ins...,A. 1\nB. 2\nC. 8\nD. 14\nE. 110,D,1.41,-0.44
11,M135401,A cat lies crouched on level ground 50 feet aw...,A. 32\nB. 38\nC. 42\nD. 60\nE. 65,C,1.19,1.15
12,M180601,(16^(1/2))^3 = ?,A. 512\nB. 64\nC. 48\nD. 24\nE. 12,B,1.76,0.84
13,M135301,The town of Mayville taxes property at a rate ...,"A. $42,000\nB. $47,250\nC. $125,000\nD. $220,5...",C,1.78,0.55


In [25]:
frames = [df1, df2]
result = pd.concat(frames)
result = result.drop_duplicates()
result.to_csv('total_naep_complete_2.csv', index=False)